In [ ]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA

include authors and title? and maybe only books 3 stars and over?

In [ ]:
read = pd.read_feather('Feather/read.feather')
books = pd.read_feather('Feather/books.feather')    

In [ ]:
def drop_empty_rows(df, column_name):
   
    df_cleaned = df.dropna(subset=[column_name])
    return df_cleaned

books = drop_empty_rows(books, 'description')

In [ ]:
books = books[books['language_code'] == 'eng'].sample(n=1000, random_state=42)

In [ ]:
books = books.reset_index(drop=True)

for all book_ids in read should be the ones in books but ill do that later

In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
from tqdm import tqdm
tqdm.pandas()
books['embeddings'] = books['combined_text'].progress_apply(lambda x: model.encode(x))

In [ ]:
embedding_matrix = np.vstack(books['embeddings'].values)

In [ ]:
print(embedding_matrix.shape)

In [ ]:
pca = PCA(n_components=50)
embedding_matrix = pca.fit_transform(embedding_matrix)

In [ ]:
cosine_sim = cosine_similarity(embedding_matrix, embedding_matrix)

In [ ]:
books['book_id']

given book get top 5 most similar

In [ ]:
def get_recommendations(book_id, books_df, cosine_sim):
    # Check if book_id is in books_df
    if book_id not in books_df['book_id'].values:
        print(f"Book ID {book_id} not found in the books dataframe.")
        return pd.DataFrame(columns=['title', 'authors'])

    # Get the index of the given book_id
    book_idx = books_df[books_df['book_id'] == book_id].index[0]

    # Calculate similarity scores
    sim_scores = list(enumerate(cosine_sim[book_idx]))

    # Sort books based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the book_ids of the top 5 most similar books
    top_book_ids = [books_df['book_id'].iloc[i[0]] for i in sim_scores[1:6]]

    # Retrieve the details of the top 5 most similar books
    top_books = books_df[books_df['book_id'].isin(top_book_ids)]

    return top_books[['title']]


In [ ]:
book_id = 17279511	
recommendations = get_recommendations(book_id, books, cosine_sim)
print(recommendations)

In [ ]:
# def get_top_rated_books(books_df, interactions_df):
#     top_rated_books = interactions_df.groupby('book_id')['rating'].mean().sort_values(ascending=False).head(5)
#     top_rated_books_df = pd.merge(books_df, top_rated_books.reset_index(), on='book_id')
#     return top_rated_books_df[['title', 'authors']]

given user get to 5 most similar books based on all books read

In [ ]:
# def get_user_recommendations(user_id, read_df, book_df, cosine_sim, num_recommendations=10):
#     # Check if user_id exists in read_df
#     if user_id not in read_df['user_id'].values:
#         print(f"User {user_id} not found in the read dataframe.")
#         return pd.DataFrame(columns=['book_id', 'title', 'authors'])

#     # Books read by the user
#     user_books = read_df[(read_df['user_id'] == user_id) & (read_df['is_read'] == 1)]['book_id'].values

#     if len(user_books) == 0:
#         print(f"User {user_id} has not read any books.")
#         return pd.DataFrame(columns=['book_id', 'title', 'authors'])

#     # Calculate similarity scores for all books
#     read_books_indices = [book_df.index[book_df['book_id'] == book_id].tolist()[0] for book_id in user_books if book_id in book_df['book_id'].values]

#     if len(read_books_indices) == 0:
#         print(f"No valid book indices found for user {user_id}.")
#         return pd.DataFrame(columns=['book_id', 'title', 'authors'])

#     # Calculate average similarity scores for books read by the user
#     sim_scores = cosine_sim[read_books_indices].mean(axis=0)

#     # Filter out books that the user has already read
#     unread_books = book_df[~book_df['book_id'].isin(user_books)].copy()

#     if unread_books.empty:
#         print(f"No unread books found for user {user_id}.")
#         return pd.DataFrame(columns=['book_id', 'title', 'authors'])

#     # Ensure unread_books indices are within bounds
#     valid_unread_books_indices = unread_books.index.values[unread_books.index.values < len(sim_scores)]
    
#     if len(valid_unread_books_indices) == 0:
#         print(f"No valid unread book indices found for user {user_id}.")
#         return pd.DataFrame(columns=['book_id', 'title', 'authors'])

#     unread_books.loc[valid_unread_books_indices, 'similarity'] = [sim_scores[i] for i in valid_unread_books_indices]

#     # Get top similar unread books
#     top_books = unread_books.sort_values(by='similarity', ascending=False).head(num_recommendations)
    
#     return top_books['book_id']